# `virtual`: Taxi Demo

In [1]:
%load_ext autoreload
%autoreload 2

import virtual

# parquet_filepath = 'https://blobs.duckdb.org/data/taxi_2019_04.parquet'
parquet_filepath = '../../demo/data/taxi_2019_04.parquet'

### One-shot table virtualization

In [2]:
# Save in Parquet format.
virtual.to_format(parquet_filepath, 'taxi_virtual.parquet', schema='./demo-debug/schema_taxi_2019_04.json', prefix='demo-debug/')

Drilling functions..
{'num': {'indices': [3, 4, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17], 'names': ['passenger_count', 'trip_distance', 'pickup_location_id', 'dropoff_location_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge']}, 'date': {'indices': [], 'names': []}, 'timestamp': {'indices': [1, 2], 'names': ['pickup_at', 'dropoff_at']}, 'time': {'indices': [], 'names': []}, 'string': {'indices': [0, 5, 6, 9], 'names': ['vendor_id', 'rate_code_id', 'store_and_fwd_flag', 'payment_type']}, 'boolean': {'indices': [], 'names': []}}
12
We found 9 function(s) in your table.
[{'target_index': 3, 'target_name': 'passenger_count', 'target_stats': {'mean': np.float64(1.5792), 'max': 6.0, 'min': 0.0}, 'models': {'sparse-lr': {'mse': 1.2183282275337988, 'intercept': np.float64(1.7884288420851584), 'coeffs': [{'col-index': 11, 'col-name': 'extra', 'coeff': np.float64(-0.1743210515185656)}]}, '2-regression': {'mse': 0.48

### Compare to vanilla `Parquet`

In [3]:
import os

print(f'Parquet: {os.path.getsize(parquet_filepath) / 1_000_000} MB')
print(f'Virtual: {os.path.getsize('taxi_virtual.parquet') / 1_000_000} MB')

Parquet: 127.056503 MB
Virtual: 100.322471 MB


### Query the vanilla Parquet file

In [4]:
import duckdb

duckdb.sql(f"select avg(total_amount) from read_parquet('{parquet_filepath}')").df()

,avg(total_amount)
0,19.229813


### Query the virtualized Parquet file

In [5]:
virtual.query(
  'select avg(total_amount) from read_parquet("taxi_virtual.parquet")',
  engine = 'duckdb'
)

,"avg((CASE WHEN ((total_amount_switch = 0)) THEN (round(((((1.0033 * fare_amount) + (1.0016 * tip_amount)) + (1.6031 * tolls_amount)) + 3.9712), 2)) ELSE round((((((1.0007 * fare_amount) + (1.0035 * tip_amount)) + (1.0056 * tolls_amount)) + (0.9389 * congestion_surcharge)) + 0.9557), 2) END + total_amount_offset))"
0,19.229813
